In [195]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [196]:
house_dict = []

for page in range(17, 21):
    # Filter Out Coming Soon, Auction, Home Type: Houses, Days On Zillow: 90 Days
    linked = 'https://www.zillow.com/ca/houses/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.9521558125,"east":-107.6611401875,"south":28.570940002914515,"north":45.333552921012235},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":5,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
#     user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
#             print('House_website:', house_website);
#             Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
#             print('listing_type:', listing_type);
            price = result.find('div', class_="list-card-price").text
            print('price:', price);
            full_address = result.find('address', class_="list-card-addr").text
#             print('full_address:', full_address);
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
#             print('street:', street);
            city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
#             print('city:', city);
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
#                 print('zip_code:', zip_code);
            except IndexError:
                zip_code = 'null'
#                 print('zip_code:', zip_code);
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
#             print('bed:', bed);
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
#             print('bath:', bath);
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
#             print('house_square_feet:', house_square_feet);
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
#             print(loop_soup)
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
#             print('lot:', lot)
#             print('type:', house_type);
#             print('year-build:', year_build);
#             print('parking:', parking);
#             print('HOA:', hoa);
#             print("Price/sqft:", zillow_price_per_sqft);

            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
            print('lot_conversion_to_sqft:', lot_conversion_to_sqft);
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
#                 print('state:', state);
            except IndexError:
                state = 'null'
            try:
                zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[0].text
                if (zestimate == "Zestimate®"):
                    zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[1].text
            except:
                zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = 'null'
#             print(zestimate)
            
            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
           }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(2)
            
            
        except AttributeError as e:
            continue

print(house_dict)    

price: $525,000
lot_conversion_to_sqft: 25700.4
price: $154,700
lot_conversion_to_sqft: 7840.0
price: $685,000
lot_conversion_to_sqft: 7470.0
price: $180,000
lot_conversion_to_sqft: 3497.0
price: $289,900
lot_conversion_to_sqft: 87120.0
price: $150,000
lot_conversion_to_sqft: 21344.4
price: $449,000
lot_conversion_to_sqft: 12632.4
price: $495,000
lot_conversion_to_sqft: 10454.0
price: $275,000
lot_conversion_to_sqft: 5000.0
price: $3,500,000
lot_conversion_to_sqft: 43560.0
price: $799,000
lot_conversion_to_sqft: 8816.0
price: $384,997
lot_conversion_to_sqft: 5601.0
price: $35,000,000
lot_conversion_to_sqft: 71874.0
price: $289,900
lot_conversion_to_sqft: 543628.8
price: $299,000
lot_conversion_to_sqft: 5519.0
price: $825,000
lot_conversion_to_sqft: 5610.0
price: $375,000
lot_conversion_to_sqft: 343252.8
price: $799,900
lot_conversion_to_sqft: 89733.6
price: $338,000
lot_conversion_to_sqft: 7200.0
price: $599,900
lot_conversion_to_sqft: 6176.0
price: $1,250,000
lot_conversion_to_sqft: 2

In [197]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,HOA,House_type,House_website,Listing_type,Lot,Lot_conversion_to_sqft,Parking,Price,Rent_zestimate,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,2,4,Riverside,"16050 Rancho Viejo Dr, Riverside, CA 92506",None,Single Family,https://www.zillow.com/homedetails/16050-Ranch...,House for sale,0.59 acres,25700.4,2 spaces,"$525,000","$2,300/mo","1,804",CA,16050 Rancho Viejo Dr,1975,"$525,001",$291,92506
1,1.5,3,Ridgecrest,"204 N Florence St, Ridgecrest, CA 93555",null,Single Family,https://www.zillow.com/homedetails/204-N-Flore...,House for sale,"7,840 sqft",7840.0,2 spaces,"$154,700","$1,350/mo","1,643",CA,204 N Florence St,1956,"$154,700",$94,93555
2,2,2,Arcadia,"6637 N Golden West Ave, Arcadia, CA 91007",None,Single Family,https://www.zillow.com/homedetails/6637-N-Gold...,House for sale,"7,470 sqft",7470.0,2 spaces,"$685,000","$2,800/mo","1,340",CA,6637 N Golden West Ave,1961,"$698,979",$511,91007
3,2,3,CanyonCountry,"20062 Crestview Dr, Canyon Country, CA 91351",None,Single Family,https://www.zillow.com/homedetails/20062-Crest...,House for sale,"3,497 sqft",3497.0,2 spaces,"$180,000","$2,700/mo","1,482",CA,20062 Crestview Dr,2001,"$180,001",$121,91351
4,1,3,Shingletown,"34700 Coot Ln, Shingletown, CA 96088",null,Single Family,https://www.zillow.com/homedetails/34700-Coot-...,House for sale,2 acres,87120.0,3 spaces,"$289,900","$1,300/mo","1,546",CA,34700 Coot Ln,2004,"$285,652",$188,96088
5,2,2,Visalia,"35599 Road 150 APT A, Visalia, CA 93292",null,Single Family,https://www.zillow.com/homedetails/35599-Road-...,House for sale,0.49 acres,21344.4,1 space,"$150,000",$825/mo,"1,440",CA,35599 Road 150 APT A,1976,"$144,209",$104,93292
6,2,4,Lancaster,"42061 Ivesgrove Dr, Lancaster, CA 93536",null,Single Family,https://www.zillow.com/homedetails/42061-Ivesg...,House for sale,0.29 acres,12632.4,2 spaces,"$449,000","$2,700/mo","2,224",CA,42061 Ivesgrove Dr,1957,"$449,836",$202,93536
7,3,4,SouthLakeTahoe,"593 Koru St, South Lake Tahoe, CA 96150",None,Single Family,https://www.zillow.com/homedetails/593-Koru-St...,House for sale,"10,454 sqft",10454.0,2 spaces,"$495,000","$2,350/mo","1,818",CA,593 Koru St,1979,"$487,457",$272,96150
8,2,3,Modesto,"1113 Vintner Ln, Modesto, CA 95358",None,Single Family,https://www.zillow.com/homedetails/1113-Vintne...,House for sale,"5,000 sqft",5000.0,2 spaces,"$275,000","$1,400/mo","1,209",CA,1113 Vintner Ln,1991,"$271,697",$227,95358
9,5,5,YorbaLinda,"4066 Live Oak Ln, Yorba Linda, CA 92886",None,Single Family,https://www.zillow.com/homedetails/4066-Live-O...,House for sale,1 acre,43560.0,3 spaces,"$3,500,000","$6,991/mo","4,900",CA,4066 Live Oak Ln,1985,"$3,390,728",$714,92886


In [198]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2020_Single_Family_Data_page17-20.csv", index=False, header=True)